# Study of a more simple trajectory
We will study a collection of 3 trajectories in order to compare the visualisation of each library

In [ ]:
"""
Function to compute the execution time.
"""
from datetime import timedelta
import time
def get_time(f, kwargs={}):
    times = []
    result = None
    for i in range(5):
        print("\nmeasure",i+1)
        t = time.time()
        result = f(**kwargs)
        res = time.time()-t
        times.append(res)
        print( timedelta(seconds=res))
        if res > 3600*13:
            print("trop de temps")
            return result
        
    print("\nall times:", times)
    a = min(times)
    b = max(times)
    print("times' outliers:",a,"&",b)
    t = abs(sum(times)-a-b)/3
    print("benchmark time",str(t)+"s","or", timedelta(seconds=t))
    return result

In [ ]:
import numpy as np
import pandas as pd
from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import movingpandas as mpd
#from holoviews import opts, dim
# import hvplot
import warnings
warnings.simplefilter("ignore")

In [ ]:
SIZE = "small"

In [ ]:
PATH="../datasets/AIS_"+SIZE+".csv"
df = pd.read_csv(PATH)
traj = mpd.TrajectoryCollection(data=df, traj_id_col='id', t='Timestamp', x='Longitude', y='Latitude')

### Outlier Detection IQR

In [ ]:
traj.add_speed(True)

In [ ]:
cleaned_traj = get_time(mpd.IqrCleaner(traj).clean,{"columns":{'speed': 3}})

In [ ]:
print("nombre precedent de points:", len(traj.to_point_gdf()))
print("nombre de points retirés:", len(traj.to_point_gdf())-len(cleaned_traj.to_point_gdf())
print("nombre actuel de points:", len(cleaned_traj.to_point_gdf()), cleaned_traj.to_point_gdf().shape)

### Outlier Detection speed Threshold

In [ ]:
cleaned_traj = get_time(mpd.OutlierCleaner(traj).clean,{"v_max":73, "units":("m", "s")})

In [ ]:
print("nombre precedent de points:", len(traj.to_point_gdf()))
print("nombre de points retirés:", len(traj.to_point_gdf())-len(cleaned_traj.to_point_gdf())
print("nombre actuel de points:", len(cleaned_traj.to_point_gdf()), cleaned_traj.to_point_gdf().shape)